# FUGIO: PHP对象注入漏洞的自动化漏洞利用生成

## 1. 摘要

PHP对象注入漏洞（POI）是一个安全关键性漏洞，它允许在存在漏洞的PHP应用程序中远程执行存在的类方法代码。利用此漏洞通常需要复杂的面向属性编程来设计注入对象。现成的工具仅专注于识别潜在的POI漏洞，而不确认漏洞利用对象的存在。为此，作者提出了 <a href="https://github.com/WSP-LAB/FUGIO" title="FUGIO: Automatic Exploit Generation for PHP Object Injection Vulnerabilities">FUGIO</a>（首个POI漏洞的自动化漏洞利用生成工具）。FUGIO进行粗粒度的静态和动态程序分析，生成了类方法链（gadgets chains），作为设计漏洞利用对象的指南。然后，FUGIO使用已识别的链中运行模糊测试，并生成漏洞利用对象。FUGIO从 30 个包含已知 POI 漏洞的应用程序中生成了 68 个漏洞利用对象，同时误报率为 0 。FUGIO 还发现了 2 个以前未报告的 POI 漏洞和 5 个漏洞利用，展示它在生成有效的漏洞利用方面的效力。

## 2. 创新点和挑战

### 2.1 创新点

作者提出了一种自动化漏洞利用生成（AEG）方法来发现 POI 漏洞，并生成利用此漏洞的注入对象。于是，设计了用于检测目标PHP应用程序中的POI漏洞，并生成漏洞利用对象来验证漏洞可利用性的FUGIO。

现有的渗透测试工具，例如 Burp 和 Acunetix ，仅专注于识别反序列化用户输入的内置调用点，报告潜在的 POI 漏洞。尽管它们根据POP链（面向属性编程，构造出一组连续的类方法调用链）产生的漏洞利用对象可以消除误报，但这需要大量的人力和专业知识。

* 静态分析可以找出有可能利用的POP链，但为了消除误报，需要解决可达性分析问题，即找到适当的输入来利用POP链。
* 随着有可能利用的POP链数量的增加，审查每条链并生成漏洞利用是一项艰巨的任务，不可能手动检查这些链并为这些链生成有效的漏洞利用。

POI漏洞的AEG方法必须实现以下目标：

1. 发现POI漏洞
2. 触发已识别漏洞的同时，识别POP链
3. 在识别的POP链中，为可利用的POP链生成输入对象，验证漏洞的可利用性

静态分析方法和上述的渗透测试工具仅仅可以找到潜在的POI漏洞，作者的核心贡献在于生成漏洞利用对象，这也是解决第二和第三个目标所必需的。

### 2.2 挑战

实现目标的技术挑战如下：

- **识别POP链**。AEG工具应该识别所有POP链，并且这些链中的方法应考虑到反序列化位置处所有可加载类的方法
- **生成漏洞利用**。AEG工具应生成具有多个属性值的漏洞利用对象，从而执行已识别链的漏洞利用。这需要生成适当的属性值，使得POP链的方法可以连续调用，最终应用程序运行到POP链尾的危险函数。其中，注入的属性值成功输入到危险函数的实参中
- **Fuzzing(模糊测试)**。尽管会带来误报的风险，作者选择更通用的**反馈驱动的fuzzing**来生成漏洞利用。而对目标PHP应用程序进行模糊测试会带来独特的技术挑战
  - 在大型 PHP 应用程序上进行无状态模糊测试时很难建立高吞吐量，即以高效率处理大量的测试输入
  - 因为需要更改多个属性值并塑造对象层次结构以启用POP链的执行，因此通过模糊测试设计多个属性值的漏洞利用对象并不简单

## 3. 概述

为了识别POP链，FUGIO进行静态和动态程序分析，以整合所有静态声明和动态生成的gadgets。然后，FUGIO进行粗粒度的过程间污点分析（目的是修建不必要的POP链），并执行深度有界的广度优先搜索，枚举所有有可能利用的链。

FUGIO通过进行反馈驱动的模糊测试来生成漏洞利用，从而解决第二个技术挑战。为了实现模糊测试的高吞吐量，FUGIO生成了一个用于测试的程序（PUT），该程序模拟触发POI漏洞的运行环境。FUGIO利用分支覆盖率、运行时引用错误，以及模糊测试期间条件表达式中出现的提示，来设计出更有可能的漏洞利用对象，从而到达被测试的POP链中更深层次的gadgets。

FUGIO由五个组件构成：POI检测器、静态分析器、动态分析器、POP链标识符和POP链fuzzer（模糊测试器）。这些组件协同工作来发现POI漏洞，并生成漏洞利用对象。

图1 描绘了FUGIO的工作流程。它采用目标PHP应用程序源代码和 URL 作为输入。POI检测器开始爬取网站以检测POI漏洞，并将检测到的漏洞传递给动态分析器。当检测到的漏洞被触发时，静态和动态分析器分别从给定的源代码和运行环境中收集数据，用于识别POP链。利用这些分析，POP链标识符计算出有可能利用的POP链，并生成PUT。对于每个已识别的链，模糊测试器会在此PUT中运行模糊测试，直到发现具体的漏洞利用。

![fugio-architecture-1](imgs/fugio-architecture-1.jpg)

<p style="text-align: center;">图1: FUGIO架构——自动化生成POI漏洞利用的工作流程</p>

## 4. 实验搭建

### 4.1 实验准备

作者在 30 个PHP应用程序中评估了FUGIO。对于每个应用程序，作者准备了一个已知的 POI 漏洞，因此要求FUGIO生成引发漏洞的漏洞利用对象，从而导致文件删除、修改和创建，命令注入或远程代码执行。在 30 个应用中，有8个应用与 Dahse 等人用于评估的应用相同。作者还纳入了 PHPGCC 的 21 个应用程序，其中12个是 PHP 库，因此使用每个库制作了一个简单的 PHP 应用程序，并向其注入了 POI 漏洞。对于其余 9 个应用程序，作者利用了已知的 POI 漏洞：CVE-2018-20148，CVE-2019-6339和<a href="https://wpscan.com/vulnerability/9567f575-529d-4d66-980c-73cba6726673" title="WooCommerce <= 3.4.5 - Authenticated Phar Deserialization">经过身份验证的Phar反序列化漏洞</a>。这些基准测试的选择标准如下：1）存在漏洞的应用版本仍然可以访问；2）存在漏洞的应用的大小并不小。

### 4.2 实验环境

作者在两颗 Intel Xeon Gold 6238 CPU @ 2.10 GHz 处理器和 384G 内存的 Linux 工作站进行实验。运行FUGIO时，作者为每个PHP版本准备了一个Docker容器，并根据对应的版本安装了Web应用程序。

## 5. 讨论、限制和未来工作

### 5.1 讨论

攻击者反序列化对象导致的安全威胁不仅存在于PHP中，还存在其他编程语言中，包括Python、Java、Ruby、Android和 .NET 。根据获得的gadgets，攻击者会实施各种恶意行为。

每种语言都有各自缓解这种威胁的方式。一个基本要点是不要直接反序列化不受信任的数据来源。因此通常建议的做法是清理用户输入。不幸的是，清理输入的逻辑应该基于反序列化方法而有所不同，但这通常会导致不正确的清理检查。

另一条建议是开发人员只允许使用原始数据类型，例如 JSON（PHP中的 json_decode 和 Python 中的 json.loads） 、YAML（.NET中的SnakeYAML 和 Python 中的 PyYAML）或 XML（Java），它们不调用反序列化回调。然而，Muñoz and Mirosh 发现， .NET 和 Java 调用 setters 方法来填充对象字段，所以它们的许多 JSON 库都是可被利用的。其他人的工作还报道了 XML 和 YAML 格式中存在类似的漏洞。

将要反序列化的类列入白名单或黑名单是一种被动缓解的方法。此方法通过利用每种语言支持的功能或者反序列化黑名单的类时引发错误来限制类的反序列化。然而，这种方法需要大量的工程成本来指定允许反序列化的类。

现有工具专注于检测不安全的反序列化。Burp Suite通过使用 <a href="https://github.com/frohoff/ysoserial" title="A proof-of-concept tool for generating payloads that exploit unsafe Java object deserialization.">ysoserial</a>（针对 Java）和 <a href="https://github.com/ambionics/phpggc" title="PHPGGC is a library of PHP unserialize() payloads along with a tool to generate them">PHPGGC</a>（针对 PHP 应用程序）发送预定义的有效载荷来检测漏洞。SerialDetector 使用污点数据流分析来识别不安全的反序列化，并通过生成有效载荷来验证已识别的漏洞。然而，它们不会将gadgets构造成有可能利用的链。有几种静态方法专注于识别可利用的gadgets链，但是它们需要手动检查的方式来消除误报。相比之下，FUGIO是一种通用的AEG工具，可以识别有可能利用的POP链，并生成漏洞利用。

### 5.2 限制

FUGIO有其局限性。FUGIO仅从目标PHP应用程序，而不是PHP内部类，提取 gadgets 构造POP链。因此它无法使用PHP内部类的 gadgets 来生成漏洞利用。生成此类漏洞需要手动将PHP的内部 gadgets 显式提供给FUGIO。出于同样的原因，FUGIO也无法利用PHP源代码转换的PHP二进制模块中的 gadgets 。

在枚举POP链时，FUGIO也没有考虑被调用者是静态不可判定的反射调用。考虑该被调用者的所有 gadgets 会导致运行模糊测试的链数量受到限制。计算该被调用者的可能值的复杂静态分析是减少漏报的另一种方法。由于模糊测试的性质，当目标链具有大量条件时，发现漏洞利用可能需要多次测试活动或更长的 timeouts（超时时间）。

### 5.3 未来工作

Dahse 等将检测到的链分为 6 种漏洞类型：文件删除（FD）、文件创建（FC）、文件修改（FM）、SQL注入（SQLi）、本地文件包含（LFI）和 XML 外部实体注入（XXE）。作者总共指定了 26 个危险函数来构造FD、FC、FM和命令注入漏洞的漏洞利用，如附录 6.1 所列。因为在PHP 5.4中无法复现，作者排除了可以引发 LFI 和 XXE 的危险函数。

尽管可以识别引发 SQLi 的POP链，但是因为此类的链需要数据库账户或已连接数据库的实例，因此模糊测试器无法到达危险函数。作者将其留给未来的工作。


## 6. 附录

### 6.1 目标危险函数

作者列出了 FUGIO 考虑的 Web 攻击涉及的危险函数，如下所示：

- 文件删除：unlink, rmdir
- 文件创建：fopen, fwrite, fputs, mkdir, copy, link, symlink, file_put_contents
- 文件修改：chmod, chown, chgrp, touch
- Shell命令注入：popen, system, passthru, exec, proc_open, shell_exec, escapeshellcmd
- 远程代码执行： eval, mail, call_user_func, call_user_func_array, preg_replace
